# Profiling

## Available profilers in Python

### lightweight
 -  **%timeit** A very usefull magic function (especially for this course!)
 -  **time** (module) This module provides various time-related functions.
 
### Standard
 -  **cProfile** (module) This module is recommended for most users; it’s a C extension with reasonable overhead that makes it suitable for profiling long-running programs. Based on lsprof, contributed by Brett Rosen and Ted Czotter. Default CPU profiler, a bit slow (deterministic)
 - **pyinstrument** Reports the call stack and elapsed times (statistical)
 - **yappi** Allows to profile multi-threaded applications (deterministic)
 - **memory_profiler** Monitors memory consumption of a process
 - **line_profiler** Profile the time individual lines of code take to execute

### Collection of Profilers
- **decoProf** Is a python tool, that bundles the above profilers and allows user's to use select the profiler they want to profile the funcion they want. https://github.com/SURFQuantum/decoProf

#### "deterministic" and "statistical" profilers:
> **_NOTE:_**  This description is taken from the decoProf documentaion https://github.com/SURFQuantum/decoProf

**Deterministic**

Deterministic profilers work by hooking into several function call/leave events and calculate all metrics according to these.

**Statistical**

Statistical profilers do not track every function call the program makes but they record the call stack every 1ms or whatever defined in the interval. The statistical profilers can impose less overhead compared to the deterministic ones.

In [4]:
import numpy as np
import time

def explicit_matmul(A,B,C):
    #A[m][n]
    #B[n][p]
    #C[m][p]    
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C[i][j] += A[i][k] * B[k][j]
    return(C)

AX=AY=BX=BY=100

A = np.random.rand(AX,AY)
B = np.random.rand(BX,BY)  
C = np.zeros((AX,AY))


start = time.perf_counter()
C_explicit = explicit_matmul(A,B,C)
end = time.perf_counter()
print("Explicit matmul: ",end-start)


Explicit matmul:  0.7024941329727881


In [5]:
import cProfile

cProfile.run('explicit_matmul(A,B,C)') #By default the run method prints to the std out


         40408 function calls in 0.871 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.854    0.854    0.871    0.871 3636018740.py:4(explicit_matmul)
    10101    0.008    0.000    0.017    0.000 <__array_function__ internals>:177(shape)
        1    0.000    0.000    0.871    0.871 <string>:1(<module>)
    10101    0.001    0.000    0.001    0.000 fromnumeric.py:1987(_shape_dispatcher)
    10101    0.002    0.000    0.002    0.000 fromnumeric.py:1991(shape)
        1    0.000    0.000    0.871    0.871 {built-in method builtins.exec}
    10101    0.005    0.000    0.007    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [6]:
cProfile.run('explicit_matmul(A,B,C)',"my_perf_file.out") #By default the run method prints to the std out

import pstats
from pstats import SortKey

p = pstats.Stats('my_perf_file.out')  #read in the profile data

#you can sort by the internal time
p.sort_stats('time')
p.print_stats()

#you can sort by the number of calls
p.sort_stats('calls')
p.print_stats()

#you can reverse the order
p.reverse_order()
p.print_stats()

Mon Nov 13 10:14:57 2023    my_perf_file.out

         40408 function calls in 0.911 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.892    0.892    0.911    0.911 /var/folders/3r/sc975sjx4zv538_w2rfj91bc0000gq/T/ipykernel_76268/3636018740.py:4(explicit_matmul)
    10101    0.009    0.000    0.019    0.000 <__array_function__ internals>:177(shape)
    10101    0.005    0.000    0.008    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    10101    0.003    0.000    0.003    0.000 /Users/benjamic/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:1991(shape)
    10101    0.002    0.000    0.002    0.000 /Users/benjamic/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:1987(_shape_dispatcher)
        1    0.000    0.000    0.911    0.911 {built-in method builtins.exec}
        1    0.000    0.000    0.911    0.911 <string>:1(<module>)
        1    0.000 

## Line Profiler
> https://pypi.org/project/line-profiler/

In [39]:
pip install line_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Coming back to the matmul example. Can you find the bug?
- You need to add the decorator @profile before each function that you would like to profile
- Call the `kernprof` script of lineprofile `kernprof -l script_to_profile.py` in order to profile your python script

In [24]:
%%writefile line_profiler_example.py
import numpy as np
import time

@profile
def explicit_matmul(A,B,C):
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C[i][j] += A[i][k] * B[k][j]
    return(C)

@profile
def inefficient_matmul(A,B,C):
    for wtf in range(3): #WTF!?
        C = np.zeros((AX,AY))
        for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
            for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
                for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                    C[i][j] += A[i][k] * B[k][j]
    return(C)


if __name__ == "__main__":

    AX=AY=BX=BY=100

    A = np.random.rand(AX,AY)
    B = np.random.rand(BX,BY)  
    C = np.zeros((AX,AY))
    
    start = time.perf_counter()
    C_explicit = explicit_matmul(A,B,C)
    end = time.perf_counter()
    print("Explicit matmul: ",end-start)
    
    
    C = np.zeros((AX,AY))
    
    start = time.perf_counter()
    C_ineff = inefficient_matmul(A,B,C)
    end = time.perf_counter()
    print("Inefficient matmul: ",end-start)
    
    
    C_ineff = inefficient_matmul(A,B,C)


Overwriting line_profiler_example.py


In [25]:
!kernprof -l line_profiler_example.py

Explicit matmul:  2.1050186720094644
Inefficient matmul:  6.093581654014997
Wrote profile results to line_profiler_example.py.lprof
Inspect results with:
python -m line_profiler -rmt "line_profiler_example.py.lprof"


**Now show the results**

In [26]:
!python -m line_profiler line_profiler_example.py.lprof

Timer unit: 1e-06 s

Total time: 1.50308 s
File: line_profiler_example.py
Function: explicit_matmul at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           @profile
     5                                           def explicit_matmul(A,B,C):
     6       101         32.0      0.3      0.0      for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
     7     10100       2452.0      0.2      0.2          for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
     8   1010000     256075.0      0.3     17.0              for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
     9   1000000    1244519.0      1.2     82.8                  C[i][j] += A[i][k] * B[k][j]
    10         1          0.0      0.0      0.0      return(C)

Total time: 8.73645 s
File: line_profiler_example.py
Function: inefficient_matmul at line 12

Line #      Hits         Time  Per Hit   % Time  Line Contents
    12                                

## Memory profiler
> https://github.com/pythonprofilers/memory_profiler

In [3]:
pip install -U memory_profiler

Note: you may need to restart the kernel to use updated packages.


In [43]:
%%writefile mem_growth.py
class simple_class:
    """A simple example class"""
    i = 100000000000000000.0
    big = bytearray(12345690)

    def f(self):
        return 'hello world'
    
@profile
def dumb():
    """This function will just keep allocating a class"""
    y = []
    for i in range(100):
        x = simple_class()
        y.append(x.i + float(i))
    return(y)

        
if __name__ == "__main__":
    dumb()


Overwriting mem_growth.py


### Get line by line memory usage

In [44]:
!python -m memory_profiler mem_growth.py

Filename: mem_growth.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9   56.734 MiB   56.734 MiB           1   @profile
    10                                         def dumb():
    11                                             """This function will just keep allocating a class"""
    12   56.734 MiB    0.000 MiB           1       y = []
    13   56.734 MiB    0.000 MiB         101       for i in range(100):
    14   56.734 MiB    0.000 MiB         100           x = simple_class()
    15   56.734 MiB    0.000 MiB         100           y.append(x.i + float(i))
    16   56.734 MiB    0.000 MiB           1       return(y)




### Plot the memory profile over time

In [30]:
!mprof run mem_growth.py
!mprof plot --output=memory.png

mprof: Sampling memory every 0.1s
running new process
running as a Python program...
Using last profile data.
